# dnotitia/Llama-DNA-1.0-8B-Instruct

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install bitsandbytes

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 1. 모델 및 토크나이저 로드
model_name = "dnotitia/Llama-DNA-1.0-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                # 4-bit 양자화
    bnb_4bit_use_double_quant=True,   # 2단계 양자화 사용 (효율성 향상)
    bnb_4bit_quant_type="nf4",        # 양자화 타입 (nf4 추천)
    bnb_4bit_compute_dtype=torch.bfloat16  # 연산 dtype (bf16 또는 float16)
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,   # 양자화 config 전달
    device_map="auto"
)


categories = {
    0: "졸업요건",
    1: "학교 공지사항",
    2: "학사일정",
    3: "식단 안내",
    4: "통학/셔틀 버스"
}


import random

def classify_query(user_query):
    """
    사용자 질의를 입력받아 5가지 카테고리 중 하나로 분류합니다.
    실패 시 무작위(0~4)로 fallback하여 절대 실패하지 않습니다.
    """

    prompt = f"""다음은 사용자 질문과 관련된 카테고리 목록입니다:
0: 졸업요건 (예: 졸업까지 몇 학점을 들어야 하나요?)
1: 학교 공지사항 (예: 이번에 올라온 공지사항 어디서 볼 수 있어요?)
2: 학사일정 (예: 이번 학기 수강신청은 언제 시작하나요?)
3: 식단 안내 (예: 오늘 학식 뭐 나와요?)
4: 통학/셔틀 버스 (예: 다음주에 셔틀버스는 정상 운행하나요?)

사용자 질문: "{user_query}"
위 질문은 어떤 카테고리에 가장 적합한가요? 숫자 레이블만 대답해주세요 (0, 1, 2, 3, 4 중 하나).

가장 적합한 카테고리 번호: """


    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=5,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=True,
        temperature=0.5
    )

    response_text = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True).strip()

    try:
        predicted_label_str = ''.join(filter(str.isdigit, response_text))
        if predicted_label_str:
            predicted_label = int(predicted_label_str[0])
            if predicted_label in categories:
                return predicted_label, categories[predicted_label]
    except Exception:
        pass

    category_examples = {
        0: "졸업 요건, 졸업 학점, 졸업 논문, 졸업 자격, 졸업 인증, 유예 신청, 졸업 필수 조건",
        1: "학교 공지사항, 안내문, 공고, 공지 업데이트, 휴강 안내, 장학금 공지, 긴급 알림",
        2: "수강 신청, 시험 기간, 성적 발표, 개강일, 종강일, 학사 일정, 수업 일정, 등록 일정",
        3: "학식, 학생 식당, 식단표, 조식, 중식, 석식, 오늘 메뉴, 급식 시간, 식단 운영",
        4: "셔틀버스, 통학버스, 운행 시간표, 버스 노선, 정류장 위치, 셔틀 예약, 통학 교통"
    }


    best_score = 0
    best_label = None
    for label, example in category_examples.items():
        score = difflib.SequenceMatcher(None, user_query, example).ratio()
        if score > best_score:
            best_score = score
            best_label = label

    return best_label, categories[best_label]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import json
from tqdm import tqdm

# 기존 classify_query 함수 사용 (import 되어 있다고 가정)

# 1. 입력 JSON 파일 로드
with open("/content/drive/MyDrive/classifier_questions_only.json", "r", encoding="utf-8") as f:
    questions_data = json.load(f)

# 2. 결과 저장 리스트
results = []

# 3. 각 질문에 대해 분류 수행
for item in tqdm(questions_data):
    question = item["question"]
    label, _ = classify_query(question)  # VARCO 모델 사용
    results.append({
        "question": question,
        "label": label
    })

# 4. 결과 JSON 파일 저장
with open("cls_output.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("✅ 분류 결과가 classified_results.json 파일로 저장되었습니다.")